In [1]:
import os
import sys
import numpy as np
import logging
import multiprocessing
from multiprocessing import Pool
import pathlib
ispec_dir = str(pathlib.Path("~/iSpec/").expanduser()) + "/"
sys.path.insert(0, os.path.abspath(ispec_dir))
import ispec

[2023-01-06 17:52:31,697] [INFO] [utils:_init_num_threads:145]: Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2023-01-06 17:52:31,699] [INFO] [utils:_init_num_threads:157]: NumExpr defaulting to 8 threads.


In [2]:
teffs = np.arange(3500, 7000.1, 250)
loggs = np.arange(3, 5.01, 0.5)
fehs = np.arange(-1, 0.51, 0.25)
alphas = np.array([0, 0.4])
grid_size = len(teffs)*len(loggs)*len(fehs)*len(alphas)

In [3]:
params = []
for t in teffs:
    for g in loggs:
        for f in fehs:
            for a in alphas:
                params.append([t, g, f, a])
params = np.array(params)

In [4]:
ranges = np.recarray((grid_size,),  dtype=[('teff', int), ('logg', float), ('MH', float), ('alpha', float), ('vmic', float)])

In [5]:
for i in range(grid_size):
    t, g, m, a = params[i]
    vmic = ispec.estimate_vmic(t, g, m)
    ranges['teff'][i] = t
    ranges['logg'][i] = g
    ranges['MH'][i] = m
    ranges['alpha'][i] = a
    ranges['vmic'][i] = vmic

In [9]:
def precompute_synthetic_grid(ranges, wnm_min, wnm_max, wavgrid_length, dir_head, code, atmosphere_model='marcs',
                              number_of_processes=1):

    precomputed_grid_dir = dir_head + "_%s/" % (code)

    ## - Read grid ranges from file
    #from astropy.io import ascii
    #ranges_filename = "input/minigrid/initial_estimate_grid_ranges.tsv"
    #ranges = ascii.read(ranges_filename, delimiter="\t")
    ## - or define them directly here (example of only 2 reference points):

    # wavelengths
    initial_wave = wnm_min
    final_wave = wnm_max
    wavelengths = np.linspace(initial_wave, final_wave, wavgrid_length)
    step_wav = np.diff(wavelengths)[0]
    res = np.median(wavelengths/step_wav)
    print ("# step wave (AA):", step_wav*10)
    print ("# resolution:", res)
    assert step_wav < 0.01
    assert res > 200000
    
    to_resolution = 400000 # Individual files will not be convolved but the grid will be (for fast comparison)
     # It can be parallelized for computers with multiple processors

    # Selected model amtosphere, linelist and solar abundances
    #model = ispec_dir + "/input/atmospheres/MARCS/"
    print ("# atmosphere model:", atmosphere_model)
    if atmosphere_model == 'marcs':
        model = ispec_dir + "/input/atmospheres/MARCS.GES/"
    else:
        model = ispec_dir + "/input/atmospheres/ATLAS9.Castelli/"
    #model = ispec_dir + "/input/atmospheres/MARCS.APOGEE/"
    #model = ispec_dir + "/input/atmospheres/ATLAS9.APOGEE/"
    #model = ispec_dir + "/input/atmospheres/ATLAS9.Castelli/"
    #model = ispec_dir + "/input/atmospheres/ATLAS9.Kurucz/"
    #model = ispec_dir + "/input/atmospheres/ATLAS9.Kirby/"

    #atomic_linelist_file = ispec_dir + "/input/linelists/transitions/VALD.300_1100nm/atomic_lines.tsv"
    atomic_linelist_file = ispec_dir + "/input/linelists/transitions/VALD.1100_2400nm/atomic_lines.tsv"
    #atomic_linelist_file = ispec_dir + "/input/linelists/transitions/GESv6_atom_hfs_iso.420_920nm/atomic_lines.tsv"
    #atomic_linelist_file = ispec_dir + "/input/linelists/transitions/GESv6_atom_nohfs_noiso.420_920nm/atomic_lines.tsv"

    isotope_file = ispec_dir + "/input/isotopes/SPECTRUM.lst"

    # Load chemical information and linelist
    atomic_linelist = ispec.read_atomic_linelist(atomic_linelist_file, wave_base=initial_wave, wave_top=final_wave)
    atomic_linelist = atomic_linelist[atomic_linelist['theoretical_depth'] >= 0.01] # Select lines that have some minimal contribution in the sun

    isotopes = ispec.read_isotope_data(isotope_file)

    if "ATLAS" in model:
        solar_abundances_file = ispec_dir + "/input/abundances/Grevesse.1998/stdatom.dat"
    else:
        # MARCS
        solar_abundances_file = ispec_dir + "/input/abundances/Grevesse.2007/stdatom.dat"
    #solar_abundances_file = ispec_dir + "/input/abundances/Asplund.2005/stdatom.dat"
    #solar_abundances_file = ispec_dir + "/input/abundances/Asplund.2009/stdatom.dat"
    #solar_abundances_file = ispec_dir + "/input/abundances/Anders.1989/stdatom.dat"

    # Load model atmospheres
    modeled_layers_pack = ispec.load_modeled_layers_pack(model)
    # Load SPECTRUM abundances
    solar_abundances = ispec.read_solar_abundances(solar_abundances_file)

    ## Custom fixed abundances
    #fixed_abundances = ispec.create_free_abundances_structure(["C", "N", "O"], chemical_elements, solar_abundances)
    #fixed_abundances['Abund'] = [-3.49, -3.71, -3.54] # Abundances in SPECTRUM scale (i.e., x - 12.0 - 0.036) and in the same order ["C", "N", "O"]
    ## No fixed abundances
    fixed_abundances = None

    ispec.precompute_synthetic_grid(precomputed_grid_dir, ranges, wavelengths, to_resolution, \
                                    modeled_layers_pack, atomic_linelist, isotopes, solar_abundances, \
                                    segments=None, number_of_processes=number_of_processes, \
                                    code=code, steps=False)

In [10]:
precompute_synthetic_grid(ranges, 1400, 1800, 120000, "ispecgrid_hband", "turbospectrum", number_of_processes=10)

# step wave (AA): 0.03333361111344857
# resolution: 479995.9999996742
# atmosphere model: marcs
[started] 3500 3.0 -1.0 0.0 1.78 0.0 0.0[started]  0.03500 0 3.0 
-1.0 0.4 1.78 0.0 0.0 0.0 [started] 03500
 3.0 -0.75 0.0 1.79 0.0 0.0 0.0[started]  03500
 3.0 -0.75 0.4 1.79 0.0[started]  0.0 35000.0  3.00 
-0.5 0.0 [started]1.8  35000.0  3.00.0  -0.50.0  0.4 01.8
 0.0 0.0[started] 0.0  35000 
3.0

[2023-01-06 17:57:05,140] [INFO] [turbospectrum:generate_spectrum:88]: Spherical model atmosphere with radius 3.65e+11 cm
[2023-01-06 17:57:05,140] [INFO] [turbospectrum:generate_spectrum:88]: Spherical model atmosphere with radius 3.65e+11 cm


 -0.25 0.0 1.81 0.0 0.0[started]  35000.0  3.00 
-0.25

[2023-01-06 17:57:05,168] [INFO] [turbospectrum:generate_spectrum:88]: Spherical model atmosphere with radius 3.65e+11 cm


 0.4 1.81 0.0 0.0[started]  0.03500 0 
3.0 0.0

...

Quick grid: 5000_4.50_-0.50_0.00_0.87_0.00_0.00_0.00.fits.gz
Quick grid: 5000_4.50_-0.50_0.40_0.87_0.00_0.00_0.00.fits.gz
Quick grid: 5000_4.50_-0.25_0.00_0.88_0.00_0.00_0.00.fits.gz
Quick grid: 5000_4.50_-0.25_0.40_0.88_0.00_0.00_0.00.fits.gz
Quick grid: 5000_4.50_0.00_0.00_0.89_0.00_0.00_0.00.fits.gz
Quick grid: 5000_4.50_0.00_0.40_0.89_0.00_0.00_0.00.fits.gz
Quick grid: 5000_4.50_0.25_0.00_0.90_0.00_0.00_0.00.fits.gz
Quick grid: 5000_4.50_0.25_0.40_0.90_0.00_0.00_0.00.fits.gz
Quick grid: 5000_4.50_0.50_0.00_0.92_0.00_0.00_0.00.fits.gz
Quick grid: 5000_4.50_0.50_0.40_0.92_0.00_0.00_0.00.fits.gz
Quick grid: 5000_5.00_-1.00_0.00_0.78_0.00_0.00_0.00.fits.gz
Quick grid: 5000_5.00_-1.00_0.40_0.78_0.00_0.00_0.00.fits.gz
Quick grid: 5000_5.00_-0.75_0.00_0.79_0.00_0.00_0.00.fits.gz
Quick grid: 5000_5.00_-0.75_0.40_0.79_0.00_0.00_0.00.fits.gz
Quick grid: 5000_5.00_-0.50_0.00_0.79_0.00_0.00_0.00.fits.gz
Quick grid: 5000_5.00_-0.50_0.40_0.79_0.00_0.00_0.00.fits.gz
Quick grid: 5000_5.00_-0.25_0.

Quick grid: 5500_4.50_-1.00_0.00_0.94_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_-1.00_0.40_0.94_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_-0.75_0.00_0.95_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_-0.75_0.40_0.95_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_-0.50_0.00_0.96_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_-0.50_0.40_0.96_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_-0.25_0.00_0.97_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_-0.25_0.40_0.97_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_0.00_0.00_0.98_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_0.00_0.40_0.98_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_0.25_0.00_0.99_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_0.25_0.40_0.99_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_0.50_0.00_1.01_0.00_0.00_0.00.fits.gz
Quick grid: 5500_4.50_0.50_0.40_1.01_0.00_0.00_0.00.fits.gz
Quick grid: 5500_5.00_-1.00_0.00_0.86_0.00_0.00_0.00.fits.gz
Quick grid: 5500_5.00_-1.00_0.40_0.86_0.00_0.00_0.00.fits.gz
Quick grid: 5500_5.00_-0.75_0.

Quick grid: 6000_4.00_0.25_0.00_1.23_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.00_0.25_0.40_1.23_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.00_0.50_0.00_1.24_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.00_0.50_0.40_1.24_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_-1.00_0.00_1.10_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_-1.00_0.40_1.10_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_-0.75_0.00_1.11_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_-0.75_0.40_1.11_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_-0.50_0.00_1.12_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_-0.50_0.40_1.12_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_-0.25_0.00_1.13_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_-0.25_0.40_1.13_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_0.00_0.00_1.14_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_0.00_0.40_1.14_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_0.25_0.00_1.15_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_0.25_0.40_1.15_0.00_0.00_0.00.fits.gz
Quick grid: 6000_4.50_0.50_0.00_

Quick grid: 6500_4.00_-0.25_0.00_1.44_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.00_-0.25_0.40_1.44_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.00_0.00_0.00_1.45_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.00_0.00_0.40_1.45_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.00_0.25_0.00_1.46_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.00_0.25_0.40_1.46_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.00_0.50_0.00_1.48_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.00_0.50_0.40_1.48_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.50_-1.00_0.00_1.34_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.50_-1.00_0.40_1.34_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.50_-0.75_0.00_1.35_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.50_-0.75_0.40_1.35_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.50_-0.50_0.00_1.36_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.50_-0.50_0.40_1.36_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.50_-0.25_0.00_1.37_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.50_-0.25_0.40_1.37_0.00_0.00_0.00.fits.gz
Quick grid: 6500_4.50_0.00_0.0

Quick grid: 7000_4.00_-0.75_0.00_1.73_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_-0.75_0.40_1.73_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_-0.50_0.00_1.74_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_-0.50_0.40_1.74_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_-0.25_0.00_1.75_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_-0.25_0.40_1.75_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_0.00_0.00_1.76_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_0.00_0.40_1.76_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_0.25_0.00_1.78_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_0.25_0.40_1.78_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_0.50_0.00_1.79_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.00_0.50_0.40_1.79_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.50_-1.00_0.00_1.65_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.50_-1.00_0.40_1.65_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.50_-0.75_0.00_1.66_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.50_-0.75_0.40_1.66_0.00_0.00_0.00.fits.gz
Quick grid: 7000_4.50_-0.50_0.

In [11]:
precompute_synthetic_grid(ranges, 1400, 1800, 120000, "ispecgrid_hband", "synthe", atmosphere_model='atlas', number_of_processes=10)

# step wave (AA): 0.03333361111344857
# resolution: 479995.9999996742
# atmosphere model: atlas
[started]  35003.0 -1.0 0.0 1.78 0.0 0.0 0.0 0
[started] 3500 3.0 -1.0 0.4 1.78 0.0 0.0 0.0[started]  35000 
3.0 -0.75 0.0 1.79 0.0 0.0 0.0 0
[started] 3500 3.0 -0.75 0.4 1.79 0.0 0.0 0.0 0[started]
 3500 3.0 -0.5 0.0 1.8 0.0 0.0 0.0 0
[started] 3500 3.0 -0.5 0.4 1.8 0.0 0.0 [started]0.0 0 
3500 3.0 -0.25 0.0 1.81 0.0 0.0 0.0 0
[started] 3500 3.0 -0.25 0.4 1.81 0.0 0.0 0.0 0
[started] 3500 3.0 0.0 0.0 1.82 0.0 0.0 0.0 0
[started] 3500 3.0 0.0 0.4 1.82 0.0 0.0 0.0 0
[finished] 3500 3.0 -1.0 0.4 1.78 0.0 0.0 0.0 0
[started] 3500 3.0 0.25 0.0 1.84 0.0 0.0 0.0 0
[finished] 3500 3.0 -0.25 0.4 1.81 0.0 0.0 0.0 0
[started] 3500 3.0 0.25 0.4 1.84 0.0 0.0 0.0 0
[finished] 3500 3.0 0.0 0.4 1.82 0.0 0.0 0.0 0
[started] 3500 3.0 0.5 0.0 1.85 0.0 0.0 0.0 0
[finished] 3500 3.0 -0.5 0.4 1.8 0.0 0.0 0.0 0
[started] 3500 3.0 0.5 0.4 1.85 0.0 0.0 0.0[finished]  03500
 3.0 -0.75 0.4 1.79 0.0 0.0 0.0 0
[started

[finished] 3750 3.0 0.25 0.0 1.65 0.0 0.0 0.0 0
[started] 3750 3.5 -0.25 0.0 1.02 0.0 0.0 0.0 0
[finished] 3750 3.5 -1.0 0.0 0.99 0.0 0.0 0.0 0
[started] 3750 3.5 -0.25 0.4 1.02 0.0 0.0 0.0 0
[finished] 3750 3.0 0.5 0.4 1.66 0.0 0.0 0.0 0
[started] 3750 3.5 0.0 0.0 1.03 0.0 0.0 0.0 0
[finished] 3750 3.0 0.5 0.0[finished]  3750 3.01.66 0.25  0.0 0.0 0.0 00.4
 1.65 0.0 0.0 0.0 0
[started] 3750 3.5 0.0 0.4 1.03 0.0 0.0 0.0 0
[started] 3750 3.5 0.25 0.0 1.04 0.0 0.0 0.0 0
[finished] 3750 3.5 -0.75 0.0 1.0 0.0 0.0 0.0 0
[started] 3750 3.5 0.25 0.4 1.04 0.0 0.0 0.0 0
[finished] 3750 3.5 -1.0 0.4 0.99 0.0 0.0 0.0 0
[started] 3750 3.5 0.5 0.0 1.06 0.0 0.0 0.0 0
[finished] 3750 3.5 -0.75 0.4 1.0 0.0 0.0 0.0 0
[started] 3750 3.5 0.5 0.4 1.06 0.0 0.0 0.0 0
[finished] 3750 3.5 -0.5 0.0 1.01 0.0 0.0 0.0 0
[started] 3750 4.0 -1.0 0.0 0.92 0.0 0.0 0.0 0
[finished] 3750 3.5 -0.5 0.4 1.01 0.0 0.0 0.0 0
[started] 3750 4.0 -1.0 0.4 0.92 0.0 0.0 0.0 0
[finished] 3750 3.5 -0.25 0.0 1.02 0.0 0.0 0.0 0
[star

[finished] 4000 4.0 -1.0 0.0 0.92 0.0 0.0 0.0 0
[started] 4000 4.0 0.0 0.0 0.96 0.0 0.0 0.0 0
[finished] 4000 3.5 0.5 0.4 1.06 0.0 0.0 0.0 0
[started] 4000 4.0 0.0 0.4 0.96 0.0 0.0 0.0 0
[finished] 4000 3.5 0.5 0.0 1.06 0.0 0.0 0.0 0
[started] 4000 4.0 0.25 0.0 0.98 0.0 0.0 0.0 0
[finished] 4000 4.0 -1.0 0.4 0.92 0.0 0.0 0.0 0
[started] 4000 4.0 0.25 0.4 0.98 0.0 0.0 0.0 0
[finished] 4000 4.0 -0.75 0.0 0.93 0.0 0.0 0.0 0
[started] 4000 4.0 0.5 0.0 0.99 0.0 0.0 0.0 0
[finished] 4000 4.0 -0.5 0.0 0.94 0.0 0.0 0.0 0
[started] 4000 4.0 0.5 0.4 0.99 0.0 0.0 0.0 0
[finished] 4000 4.0 -0.75 0.4 0.93 0.0 0.0 0.0 0
[started] 4000 4.5 -1.0 0.0 0.85 0.0 0.0 0.0 0
[finished] 4000 4.0 -0.5 0.4 0.94 0.0 0.0 0.0 0
[started] 4000 4.5 -1.0 0.4 0.85 0.0 0.0 0.0 0
[finished] 4000 4.0 -0.25 0.0 0.95 0.0 0.0 0.0 0
[started] 4000 4.5 -0.75 0.0 0.86 0.0 0.0 0.0 0
[finished] 4000 4.0 -0.25 0.4 0.95 0.0 0.0 0.0 0
[started] 4000 4.5 -0.75 0.4 0.86 0.0 0.0 0.0 0
[finished] 4000 4.0 0.0 0.0 0.96 0.0 0.0 0.0 0
[st

[finished] 4250 4.5 -1.0 0.0 0.85 0.0 0.0 0.0 0
[started] 4250 4.5 0.25 0.0 0.9 0.0 0.0 0.0 0
[finished] 4250 4.5 -1.0 0.4 0.85 0.0 0.0 0.0 0
[started] 4250 4.5 0.25 0.4 0.9 0.0 0.0 0.0 0
[finished] 4250 4.5 -0.75 0.0 0.86 0.0[finished]  42500.0 0.0  04.5
 -0.75 0.4 0.86 0.0 0.0 0.0 0
[started] 4250 4.5 0.5 0.0 0.92 0.0 0.0 [started] 0.04250  4.50 
0.5 0.4 0.92 0.0 0.0 0.0 0
[finished] 4250 4.5 -0.5 0.0 0.87 0.0 0.0 0.0 0
[started] 4250 5.0 -1.0 0.0 0.78 0.0 0.0 0.0 0
[finished] 4250 4.5 -0.5 0.4 0.87 0.0 0.0 0.0 0
[started] 4250 5.0 -1.0 0.4 0.78 0.0 0.0 0.0 0
[finished] 4250 4.5 -0.25 0.0 0.88 0.0 0.0 0.0 0
[started] 4250 5.0 -0.75 0.0 0.79 0.0 0.0 0.0 0
[finished] 4250 4.5 -0.25 0.4 0.88 0.0 0.0 0.0 0
[started] 4250 5.0 -0.75 0.4 0.79 0.0 0.0 0.0 0
[finished] 4250 4.5 0.0 0.0 0.89 0.0 0.0 0.0 0
[started] 4250 5.0 -0.5 0.0 0.79 0.0 0.0 0.0 0
[finished] 4250 4.5 0.0 0.4 0.89 0.0 0.0 0.0 0
[started] 4250 5.0 -0.5 0.4 0.79 0.0 0.0 0.0 0
[finished] 4250 4.5 0.25 0.4 0.9 0.0 0.0 0.0 0
[st

[finished] 4500 4.5 0.5 0.4 0.92 0.0 0.0 0.0 0
[started] 4500 5.0 0.5 0.0 0.84 0.0 0.0 0.0 0
[finished] 4500 5.0 -0.75 0.4 0.79 0.0 0.0 0.0 0
[started] 4500 5.0 0.5 0.4 0.84 0.0 0.0 0.0 0
[finished] 4500 5.0 -0.5 0.0 0.79 0.0 0.0 0.0 0
[started] 4750 3.0 -1.0 0.0 1.22 0.0 0.0 0.0 0
[finished] 4500 5.0 -0.25 0.0 0.81 0.0 0.0 0.0 0
[started] 4750 3.0 -1.0 0.4 1.22 0.0 0.0 0.0 0
[finished] 4500 5.0 -0.5 0.4 0.79 0.0 0.0 0.0 0
[started] 4750 3.0 -0.75 0.0 1.23 0.0 0.0 0.0 0
[finished] 4500 5.0 -0.25 0.4 0.81 0.0 0.0 0.0 0
[started] 4750 3.0 -0.75 0.4 1.23 0.0 0.0 0.0 0
[finished] 4500 5.0 0.0 0.0 0.82 0.0 0.0 0.0 0
[started] 4750 3.0 -0.5 0.0 1.24 0.0 0.0 0.0 0
[finished] 4500 5.0 0.0 0.4 0.82 0.0 0.0 0.0 0
[started] 4750 3.0 -0.5 0.4 1.24 0.0 0.0 0.0 0
[finished] 4500 5.0 0.25 0.0 0.83 0.0 0.0 0.0 0
[started] 4750 3.0 -0.25 0.0 1.25 0.0 0.0 0.0 0
[finished] 4500 5.0 0.25 0.4 0.83 0.0 0.0 0.0 0
[started] 4750 3.0 -0.25 0.4 1.25 0.0 0.0 0.0 0
[finished] 4500 5.0 0.5 0.0 0.84 0.0 0.0 0.0 0
[

...

Quick grid: 4750_3.50_0.50_0.40_1.06_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_-1.00_0.00_0.92_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_-1.00_0.40_0.92_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_-0.75_0.00_0.93_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_-0.75_0.40_0.93_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_-0.50_0.00_0.94_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_-0.50_0.40_0.94_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_-0.25_0.00_0.95_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_-0.25_0.40_0.95_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_0.00_0.00_0.96_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_0.00_0.40_0.96_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_0.25_0.00_0.98_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_0.25_0.40_0.98_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_0.50_0.00_0.99_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.00_0.50_0.40_0.99_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.50_-1.00_0.00_0.85_0.00_0.00_0.00.fits.gz
Quick grid: 4750_4.50_-1.00_0.4

Quick grid: 5250_3.50_0.00_0.40_1.07_0.00_0.00_0.00.fits.gz
Quick grid: 5250_3.50_0.25_0.00_1.08_0.00_0.00_0.00.fits.gz
Quick grid: 5250_3.50_0.25_0.40_1.08_0.00_0.00_0.00.fits.gz
Quick grid: 5250_3.50_0.50_0.00_1.09_0.00_0.00_0.00.fits.gz
Quick grid: 5250_3.50_0.50_0.40_1.09_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_-1.00_0.00_0.96_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_-1.00_0.40_0.96_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_-0.75_0.00_0.96_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_-0.75_0.40_0.96_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_-0.50_0.00_0.97_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_-0.50_0.40_0.97_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_-0.25_0.00_0.98_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_-0.25_0.40_0.98_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_0.00_0.00_1.00_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_0.00_0.40_1.00_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_0.25_0.00_1.01_0.00_0.00_0.00.fits.gz
Quick grid: 5250_4.00_0.25_0.40_

Quick grid: 5750_3.50_-0.50_0.40_1.17_0.00_0.00_0.00.fits.gz
Quick grid: 5750_3.50_-0.25_0.00_1.18_0.00_0.00_0.00.fits.gz
Quick grid: 5750_3.50_-0.25_0.40_1.18_0.00_0.00_0.00.fits.gz
Quick grid: 5750_3.50_0.00_0.00_1.19_0.00_0.00_0.00.fits.gz
Quick grid: 5750_3.50_0.00_0.40_1.19_0.00_0.00_0.00.fits.gz
Quick grid: 5750_3.50_0.25_0.00_1.20_0.00_0.00_0.00.fits.gz
Quick grid: 5750_3.50_0.25_0.40_1.20_0.00_0.00_0.00.fits.gz
Quick grid: 5750_3.50_0.50_0.00_1.22_0.00_0.00_0.00.fits.gz
Quick grid: 5750_3.50_0.50_0.40_1.22_0.00_0.00_0.00.fits.gz
Quick grid: 5750_4.00_-1.00_0.00_1.08_0.00_0.00_0.00.fits.gz
Quick grid: 5750_4.00_-1.00_0.40_1.08_0.00_0.00_0.00.fits.gz
Quick grid: 5750_4.00_-0.75_0.00_1.09_0.00_0.00_0.00.fits.gz
Quick grid: 5750_4.00_-0.75_0.40_1.09_0.00_0.00_0.00.fits.gz
Quick grid: 5750_4.00_-0.50_0.00_1.10_0.00_0.00_0.00.fits.gz
Quick grid: 5750_4.00_-0.50_0.40_1.10_0.00_0.00_0.00.fits.gz
Quick grid: 5750_4.00_-0.25_0.00_1.11_0.00_0.00_0.00.fits.gz
Quick grid: 5750_4.00_-0.25_0.

Quick grid: 6250_3.50_-1.00_0.40_1.35_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_-0.75_0.00_1.36_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_-0.75_0.40_1.36_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_-0.50_0.00_1.37_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_-0.50_0.40_1.37_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_-0.25_0.00_1.38_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_-0.25_0.40_1.38_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_0.00_0.00_1.39_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_0.00_0.40_1.39_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_0.25_0.00_1.40_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_0.25_0.40_1.40_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_0.50_0.00_1.42_0.00_0.00_0.00.fits.gz
Quick grid: 6250_3.50_0.50_0.40_1.42_0.00_0.00_0.00.fits.gz
Quick grid: 6250_4.00_-1.00_0.00_1.28_0.00_0.00_0.00.fits.gz
Quick grid: 6250_4.00_-1.00_0.40_1.28_0.00_0.00_0.00.fits.gz
Quick grid: 6250_4.00_-0.75_0.00_1.29_0.00_0.00_0.00.fits.gz
Quick grid: 6250_4.00_-0.75_0.

Quick grid: 6750_3.00_0.25_0.40_2.38_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.00_0.50_0.00_2.40_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.00_0.50_0.40_2.40_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_-1.00_0.00_1.63_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_-1.00_0.40_1.63_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_-0.75_0.00_1.64_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_-0.75_0.40_1.64_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_-0.50_0.00_1.64_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_-0.50_0.40_1.64_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_-0.25_0.00_1.66_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_-0.25_0.40_1.66_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_0.00_0.00_1.67_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_0.00_0.40_1.67_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_0.25_0.00_1.68_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_0.25_0.40_1.68_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_0.50_0.00_1.69_0.00_0.00_0.00.fits.gz
Quick grid: 6750_3.50_0.50_0.40_